In [2]:
# Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import plotly.graph_objs as go
from plotly.subplots import make_subplots
from scipy import stats
from scipy.stats import skew, boxcox
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from category_encoders import TargetEncoder
from sklearn.impute import KNNImputer
from sklearn.ensemble import RandomForestRegressor

import warnings
warnings.filterwarnings("ignore")

In [3]:
cell_line_details = pd.read_excel('/public/home/liujunwu/workdir/DataSets/Kaggle/GDSC/Cell_Lines_Details.xlsx',sheet_name='Cell line details')
compound_anno = pd.read_csv('/public/home/liujunwu/workdir/DataSets/Kaggle/GDSC/Compounds-annotation.csv',sep=',')
gdsc_data = pd.read_csv('/public/home/liujunwu/workdir/DataSets/Kaggle/GDSC/GDSC2-dataset.csv') 


In [4]:
#cell_line_details['COSMIC identifier'] = cell_line_details['COSMIC identifier'].astype('int64')
cell_line_details.head()

,Sample Name,COSMIC identifier,Whole Exome Sequencing (WES),Copy Number Alterations (CNA),Gene Expression,Methylation,Drug\nResponse,GDSC\nTissue descriptor 1,GDSC\nTissue\ndescriptor 2,Cancer Type\n(matching TCGA label),Microsatellite \ninstability Status (MSI),Screen Medium,Growth Properties
0,A253,906794,Y,Y,Y,Y,Y,aero_dig_tract,head and neck,NaN,MSS/MSI-L,D/F12,Adherent
1,BB30-HNC,753531,Y,Y,Y,Y,Y,aero_dig_tract,head and neck,HNSC,MSS/MSI-L,D/F12,Adherent
2,BB49-HNC,753532,Y,Y,Y,Y,Y,aero_dig_tract,head and neck,HNSC,MSS/MSI-L,D/F12,Adherent
3,BHY,753535,Y,Y,Y,Y,Y,aero_dig_tract,head and neck,HNSC,MSS/MSI-L,D/F12,Adherent
4,BICR10,1290724,Y,Y,Y,Y,Y,aero_dig_tract,head and neck,HNSC,MSS/MSI-L,D/F12,Adherent


In [5]:
cell_line_details['Growth Properties'].unique()

array(['Adherent', 'Semi-Adherent', 'Suspension', nan], dtype=object)

In [13]:
compound_anno = compound_anno[['DRUG_ID', 'DRUG_NAME','TARGET', 'TARGET_PATHWAY']] ## drop SCREENING_SITE 
compound_anno.head()

,DRUG_ID,DRUG_NAME,TARGET,TARGET_PATHWAY
0,1,Erlotinib,EGFR,EGFR signaling
1,3,Rapamycin,MTORC1,PI3K/MTOR signaling
2,5,Sunitinib,"PDGFR, KIT, VEGFR, FLT3, RET, CSF1R",RTK signaling
3,6,PHA-665752,MET,RTK signaling
4,9,MG-132,"Proteasome, CAPN1",Protein stability and degradation


In [6]:
gdsc_data.head()

,DATASET,NLME_RESULT_ID,NLME_CURVE_ID,COSMIC_ID,CELL_LINE_NAME,SANGER_MODEL_ID,TCGA_DESC,DRUG_ID,DRUG_NAME,PUTATIVE_TARGET,PATHWAY_NAME,COMPANY_ID,WEBRELEASE,MIN_CONC,MAX_CONC,LN_IC50,AUC,RMSE,Z_SCORE
0,GDSC2,343,15946310,683667,PFSK-1,SIDM01132,MB,1003,Camptothecin,TOP1,DNA replication,1046,Y,0.0001,0.1,-1.463887,0.930220,0.089052,0.433123
1,GDSC2,343,15946548,684052,A673,SIDM00848,UNCLASSIFIED,1003,Camptothecin,TOP1,DNA replication,1046,Y,0.0001,0.1,-4.869455,0.614970,0.111351,-1.421100
2,GDSC2,343,15946830,684057,ES5,SIDM00263,UNCLASSIFIED,1003,Camptothecin,TOP1,DNA replication,1046,Y,0.0001,0.1,-3.360586,0.791072,0.142855,-0.599569
3,GDSC2,343,15947087,684059,ES7,SIDM00269,UNCLASSIFIED,1003,Camptothecin,TOP1,DNA replication,1046,Y,0.0001,0.1,-5.044940,0.592660,0.135539,-1.516647
4,GDSC2,343,15947369,684062,EW-11,SIDM00203,UNCLASSIFIED,1003,Camptothecin,TOP1,DNA replication,1046,Y,0.0001,0.1,-3.741991,0.734047,0.128059,-0.807232


In [7]:
gdsc_data = gdsc_data[['COSMIC_ID', 'CELL_LINE_NAME', 'TCGA_DESC', 'DRUG_ID', 'PUTATIVE_TARGET', 'PATHWAY_NAME', 'LN_IC50', 'AUC', 'Z_SCORE']]
gdsc_data.head()


,COSMIC_ID,CELL_LINE_NAME,TCGA_DESC,DRUG_ID,PUTATIVE_TARGET,PATHWAY_NAME,LN_IC50,AUC,Z_SCORE
0,683667,PFSK-1,MB,1003,TOP1,DNA replication,-1.463887,0.930220,0.433123
1,684052,A673,UNCLASSIFIED,1003,TOP1,DNA replication,-4.869455,0.614970,-1.421100
2,684057,ES5,UNCLASSIFIED,1003,TOP1,DNA replication,-3.360586,0.791072,-0.599569
3,684059,ES7,UNCLASSIFIED,1003,TOP1,DNA replication,-5.044940,0.592660,-1.516647
4,684062,EW-11,UNCLASSIFIED,1003,TOP1,DNA replication,-3.741991,0.734047,-0.807232


In [8]:
#1.1. Merging Tables
merged_data = pd.merge(gdsc_data, cell_line_details, how='left',left_on='COSMIC_ID', right_on = 'COSMIC identifier')
merged_data.head()

,COSMIC_ID,CELL_LINE_NAME,TCGA_DESC,DRUG_ID,PUTATIVE_TARGET,PATHWAY_NAME,LN_IC50,AUC,Z_SCORE,Sample Name,...,Copy Number Alterations (CNA),Gene Expression,Methylation,Drug\nResponse,GDSC\nTissue descriptor 1,GDSC\nTissue\ndescriptor 2,Cancer Type\n(matching TCGA label),Microsatellite \ninstability Status (MSI),Screen Medium,Growth Properties
0,683667,PFSK-1,MB,1003,TOP1,DNA replication,-1.463887,0.930220,0.433123,PFSK-1,...,Y,Y,Y,Y,nervous_system,medulloblastoma,MB,MSS/MSI-L,R,Adherent
1,684052,A673,UNCLASSIFIED,1003,TOP1,DNA replication,-4.869455,0.614970,-1.421100,A673,...,Y,Y,Y,Y,soft_tissue,rhabdomyosarcoma,NaN,MSS/MSI-L,D/F12,Adherent
2,684057,ES5,UNCLASSIFIED,1003,TOP1,DNA replication,-3.360586,0.791072,-0.599569,ES5,...,Y,Y,Y,Y,bone,ewings_sarcoma,NaN,MSS/MSI-L,R,Adherent
3,684059,ES7,UNCLASSIFIED,1003,TOP1,DNA replication,-5.044940,0.592660,-1.516647,ES7,...,Y,Y,Y,Y,bone,ewings_sarcoma,NaN,MSS/MSI-L,R,Adherent
4,684062,EW-11,UNCLASSIFIED,1003,TOP1,DNA replication,-3.741991,0.734047,-0.807232,EW-11,...,Y,Y,Y,Y,bone,ewings_sarcoma,NaN,MSS/MSI-L,R,Adherent


In [26]:
final_data = pd.merge(merged_data, compound_anno, on='DRUG_ID', how='left')
final_data.head()

,COSMIC_ID,CELL_LINE_NAME,TCGA_DESC,DRUG_ID,PUTATIVE_TARGET,PATHWAY_NAME,LN_IC50,AUC,Z_SCORE,Sample Name,...,GDSC\nTissue\ndescriptor 2,Cancer Type\n(matching TCGA label),Microsatellite \ninstability Status (MSI),Screen Medium,Growth Properties,SCREENING_SITE,DRUG_NAME,SYNONYMS,TARGET,TARGET_PATHWAY
0,683667,PFSK-1,MB,1003,TOP1,DNA replication,-1.463887,0.930220,0.433123,PFSK-1,...,medulloblastoma,MB,MSS/MSI-L,R,Adherent,SANGER,Camptothecin,"Camptothecine, (+)-Camptothecin",TOP1,DNA replication
1,684052,A673,UNCLASSIFIED,1003,TOP1,DNA replication,-4.869455,0.614970,-1.421100,A673,...,rhabdomyosarcoma,NaN,MSS/MSI-L,D/F12,Adherent,SANGER,Camptothecin,"Camptothecine, (+)-Camptothecin",TOP1,DNA replication
2,684057,ES5,UNCLASSIFIED,1003,TOP1,DNA replication,-3.360586,0.791072,-0.599569,ES5,...,ewings_sarcoma,NaN,MSS/MSI-L,R,Adherent,SANGER,Camptothecin,"Camptothecine, (+)-Camptothecin",TOP1,DNA replication
3,684059,ES7,UNCLASSIFIED,1003,TOP1,DNA replication,-5.044940,0.592660,-1.516647,ES7,...,ewings_sarcoma,NaN,MSS/MSI-L,R,Adherent,SANGER,Camptothecin,"Camptothecine, (+)-Camptothecin",TOP1,DNA replication
4,684062,EW-11,UNCLASSIFIED,1003,TOP1,DNA replication,-3.741991,0.734047,-0.807232,EW-11,...,ewings_sarcoma,NaN,MSS/MSI-L,R,Adherent,SANGER,Camptothecin,"Camptothecine, (+)-Camptothecin",TOP1,DNA replication


In [28]:
final_data.head(2).T

,0,1
COSMIC_ID,683667,684052
CELL_LINE_NAME,PFSK-1,A673
TCGA_DESC,MB,UNCLASSIFIED
DRUG_ID,1003,1003
PUTATIVE_TARGET,TOP1,TOP1
PATHWAY_NAME,DNA replication,DNA replication
LN_IC50,-1.463887,-4.869455
AUC,0.93022,0.61497
Z_SCORE,0.433123,-1.4211
Sample Name,PFSK-1,A673


In [ ]:
final_data.columns = final_data.columns.str.replace('\n', ' ') ## 去除列名中的换行符


In [43]:
final_data.describe(include=[np.number]).transpose()


,count,mean,std,min,25%,50%,75%,max
COSMIC_ID,242035.0,992107.148487,220981.426065,683667.000000,906805.000000,909720.000000,1.240144e+06,1.789883e+06
DRUG_ID,242035.0,1594.044886,398.739728,1003.000000,1149.000000,1631.000000,1.912000e+03,2.499000e+03
LN_IC50,242035.0,2.817111,2.762191,-8.747724,1.508054,3.236744,4.700111e+00,1.382019e+01
AUC,242035.0,0.882593,0.146998,0.006282,0.849452,0.944197,9.749340e-01,9.989040e-01
Z_SCORE,242035.0,0.000006,0.999390,-8.254501,-0.656842,0.010585,6.560365e-01,7.978776e+00


In [32]:
print (final_data.shape)
final_data.columns = final_data.columns.str.replace('\n', ' ') ## 去除列名中的换行符
final_data = final_data.drop(columns=['PUTATIVE_TARGET','PATHWAY_NAME','COSMIC identifier','Sample Name'], index=1) ## 去除重复行
print (final_data.shape)


(242036, 27)
(242035, 23)


In [33]:
final_data.to_csv('/public/home/liujunwu/workdir/DataSets/Kaggle/GDSC/final_data.csv', index=False,sep='\t')

In [10]:
# Check the shape of the final dataset
print(final_data.shape)

(242036, 27)


In [42]:
## 一个药物名对应多个药物ID的情形
drug_mapping = final_data[['DRUG_ID', 'DRUG_NAME']].drop_duplicates()
duplicates = drug_mapping[drug_mapping.duplicated('DRUG_NAME', keep=False)]
print(duplicates)


        DRUG_ID     DRUG_NAME
3212       1007     Docetaxel
42959      1062   Selumetinib
55178      1089   Oxaliplatin
69109      1200   Fulvestrant
90529      1553    Uprosertib
117542     1627        GSK343
133166     1736   Selumetinib
144335     1803      Acetalax
145052     1804      Acetalax
145769     1806   Oxaliplatin
148637     1811  Dactinomycin
152221     1816   Fulvestrant
153655     1819     Docetaxel
177231     1908   Ulixertinib
180104     1911  Dactinomycin
207847     2037        GSK343
215011     2047   Ulixertinib
218768     2106    Uprosertib


In [12]:
drug_id_map_name[drug_id_map_name['DRUG_NAME'] == 'Selumetinib']

,DRUG_ID,DRUG_NAME
42959,1062,Selumetinib
133166,1736,Selumetinib


In [17]:
numeric_features = final_data.select_dtypes(include=[np.number])
final_data.describe(include=[np.number]).transpose()

,count,mean,std,min,25%,50%,75%,max
NLME_RESULT_ID,242036.0,3.430000e+02,0.000000,3.430000e+02,3.430000e+02,3.430000e+02,3.430000e+02,3.430000e+02
NLME_CURVE_ID,242036.0,1.606806e+07,70287.494451,1.594631e+07,1.600719e+07,1.606807e+07,1.612893e+07,1.618978e+07
COSMIC_ID,242036.0,9.921059e+05,220981.856696,6.836670e+05,9.068050e+05,9.097200e+05,1.240144e+06,1.789883e+06
DRUG_ID,242036.0,1.594042e+03,398.740714,1.003000e+03,1.149000e+03,1.631000e+03,1.912000e+03,2.499000e+03
COMPANY_ID,242036.0,1.042967e+03,16.911327,1.001000e+03,1.043000e+03,1.046000e+03,1.046000e+03,1.101000e+03
MIN_CONC,242036.0,2.314284e-02,0.158738,1.000000e-05,3.002000e-03,1.000500e-02,1.000500e-02,2.001054e+00
MAX_CONC,242036.0,2.346228e+01,158.622810,1.000000e-02,3.000000e+00,1.000000e+01,1.000000e+01,2.000000e+03
LN_IC50,242036.0,2.817079e+00,2.762229,-8.747724e+00,1.508018e+00,3.236731e+00,4.700110e+00,1.382019e+01
AUC,242036.0,8.825921e-01,0.146998,6.282000e-03,8.494487e-01,9.441965e-01,9.749340e-01,9.989040e-01
RMSE,242036.0,8.277921e-02,0.042695,3.274000e-03,5.110675e-02,7.608300e-02,1.061053e-01,2.999840e-01


In [ ]:
Data = final_data
categorical_features = Data.select_dtypes(include=object)
Data.describe(include=object).transpose()

In [35]:
# Data Preprocessing
final_data.head().T

,0,2,3,4,5
COSMIC_ID,683667,684057,684059,684062,684072
CELL_LINE_NAME,PFSK-1,ES5,ES7,EW-11,SK-ES-1
TCGA_DESC,MB,UNCLASSIFIED,UNCLASSIFIED,UNCLASSIFIED,UNCLASSIFIED
DRUG_ID,1003,1003,1003,1003,1003
LN_IC50,-1.463887,-3.360586,-5.04494,-3.741991,-5.142961
AUC,0.93022,0.791072,0.59266,0.734047,0.582439
Z_SCORE,0.433123,-0.599569,-1.516647,-0.807232,-1.570016
Whole Exome Sequencing (WES),Y,Y,Y,Y,Y
Copy Number Alterations (CNA),Y,Y,Y,Y,Y
Gene Expression,Y,Y,Y,Y,Y


In [37]:
final_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 242035 entries, 0 to 242035
Data columns (total 23 columns):
 #   Column                                    Non-Null Count   Dtype  
---  ------                                    --------------   -----  
 0   COSMIC_ID                                 242035 non-null  int64  
 1   CELL_LINE_NAME                            242035 non-null  object 
 2   TCGA_DESC                                 240968 non-null  object 
 3   DRUG_ID                                   242035 non-null  int64  
 4   LN_IC50                                   242035 non-null  float64
 5   AUC                                       242035 non-null  float64
 6   Z_SCORE                                   242035 non-null  float64
 7   Whole Exome Sequencing (WES)              242035 non-null  object 
 8   Copy Number Alterations (CNA)             242035 non-null  object 
 9   Gene Expression                           242035 non-null  object 
 10  Methylation              

In [39]:
final_data.describe(include=object).transpose()


,count,unique,top,freq
CELL_LINE_NAME,242035,969,SW620,295
TCGA_DESC,240968,32,UNCLASSIFIED,45690
Whole Exome Sequencing (WES),242035,1,Y,242035
Copy Number Alterations (CNA),242035,2,Y,240847
Gene Expression,242035,2,Y,236791
Methylation,242035,2,Y,233917
Drug Response,242035,2,Y,240609
GDSC Tissue descriptor 1,242035,19,lung_NSCLC,27147
GDSC Tissue descriptor 2,242035,55,lung_NSCLC_adenocarcinoma,16282
Cancer Type (matching TCGA label),198523,31,LUAD,15653


In [40]:
final_data['Copy Number Alterations (CNA)'].unique()

array(['Y', 'N'], dtype=object)